In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/summarize/rouge.py
/kaggle/input/summarize/clef_taskC_test3_full.json
/kaggle/input/summarize/train.json
/kaggle/input/summarize/valid.json
/kaggle/input/sec-edgar-companies-list/sec__edgar_company_info.csv
/kaggle/input/sec-edgar-companies-list/database.sqlite


In [1]:
# !pip install transformers
!pip install accelerate
# !pip install datasets
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 5.7 MB/s eta 0:00:0000:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.20.3
    Uninstalling accelerate-0.20.3:
      Successfully uninstalled accelerate-0.20.3
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=3abaf913428f3ed2b0d4d96ea51c049efff0ba11b5f87f15979e06cd119c7ea4
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [2]:
max_target_length = 1024

padding = "max_length"

label_pad_token_id = -100

weight_decay = 0.01
learning_rate = 0.0001

gradient_accumulation_steps = 5
train_epoch = 10

scheduler_type = "linear"
num_warmup_steps = 5

output_dir = "/kaggle/working/"

In [7]:
def training_loop(seed: int = 42, batch_size: int = 8):

    import transformers
    from accelerate import Accelerator

    from huggingface_hub import Repository
    from transformers import (
        CONFIG_MAPPING,
        MODEL_MAPPING,
        AdamW,
        AutoConfig,
        AutoModelForSeq2SeqLM,
        AutoModelForMaskedLM,
        LEDTokenizer,
        LEDForConditionalGeneration,
        AutoTokenizer,
        AutoModel,
        EncoderDecoderConfig,
        LongformerTokenizer, 
        EncoderDecoderModel,
        BartTokenizer,
        DataCollatorForSeq2Seq,
        SchedulerType,
        get_scheduler,
        set_seed
    )
    from transformers.file_utils import get_full_repo_name, is_offline_mode
    from transformers.utils.versions import require_version
    import torch
    from torch.utils.data import DataLoader
    import numpy as np

    from datasets import load_dataset
    import math
    from datasets import load_metric
    import nltk
    from tqdm import tqdm
#     torch.multiprocessing.set_start_method('spawn')


    nltk.download("punkt", quiet=True)
    # Initialize accelerator
    accelerator = Accelerator()
    dataset_name = "cnn_dailymail"
    raw_datasets = load_dataset(dataset_name)
    
    column_names = raw_datasets["train"].column_names

    text_column = column_names[0]
    summary_column = column_names[1]

    
#     accelerate.print("AAA")

    set_seed(seed)
        
    tokenizer = BartTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")
    model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6")
    print("*"*100)
    
    def preprocess_function(examples):
        inputs = examples[text_column]
        targets = examples[summary_column]

        model_inputs = tokenizer(inputs,
                               max_length= 1024,
                               padding=padding,
                               truncation=True)

        with tokenizer.as_target_tokenizer():
            labels = tokenizer(targets,
                               max_length=max_target_length,
                               padding=padding,
                               truncation=True)

        if padding=="max_length":
            labels["input_ids"] = [
                [(l if l!=tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
            ]

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs
    
    def postprocess_text(preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [label.strip() for label in labels]

        # rougeLSum expects newline after each sentence
        preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
        labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

        return preds, labels
    
    def get_dataloader(batch_size):
    
        with accelerator.main_process_first():
            processed_datasets = raw_datasets.map(
                      preprocess_function,
                      batched=True,
                      remove_columns=column_names,
                      desc="Running tokenizer on dataset",
                  )


        train_dataset = processed_datasets["train"]
        eval_dataset = processed_datasets["validation"]
        test_dataset = processed_datasets["test"]

        data_collator = DataCollatorForSeq2Seq(
            tokenizer,
            model=model,
            label_pad_token_id=label_pad_token_id
        )

        train_dataloader = DataLoader(
            train_dataset, shuffle=True, collate_fn=data_collator, batch_size=batch_size
        )
        eval_dataloader = DataLoader(eval_dataset, collate_fn=data_collator, batch_size=batch_size)
        test_dataloader = DataLoader(test_dataset, collate_fn=data_collator, batch_size=batch_size)

        return train_dataloader, eval_dataloader, test_dataloader
    
    train_dataloader, eval_dataloader, test_dataloader = get_dataloader(batch_size=1)
    metric = load_metric("/kaggle/input/summarize/rouge.py")
    print("*"*100)
    # Optimizer
    # Split weights in two groups, one with weight decay and the other not.
    no_decay = ["bias", "LayerNorm.weight"]
    
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr= learning_rate)
    
    num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)

    max_train_steps = train_epoch * num_update_steps_per_epoch

    lr_scheduler = get_scheduler(
            name=scheduler_type,
            optimizer=optimizer,
            num_warmup_steps=num_warmup_steps,
            num_training_steps=max_train_steps,
        )
    
    print("*"*100)
    model, optimizer, train_dataloader, eval_dataloader, test_dataloader, lr_scheduler = accelerator.prepare(
        model, optimizer, train_dataloader, eval_dataloader, test_dataloader, lr_scheduler)
    
    progress_bar = tqdm(range(max_train_steps), disable=not accelerator.is_local_main_process)
    completed_steps = 0
    best_val_score = 0
    best_epoch = -1
    print("*"*100)
    for epoch in range(train_epoch):
        model.train()

        for step, batch in enumerate(train_dataloader):
            outputs = model(**batch)

            loss = outputs.loss
            loss = loss/gradient_accumulation_steps

            accelerator.backward(loss)

            if step % gradient_accumulation_steps == 0 or step == len(train_dataloader)-1:
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
                progress_bar.update(1)
                completed_steps +=1

            if completed_steps >= max_train_steps:
                break

        model.eval()
        val_max_target_length = max_target_length

        gen_kwargs = {
          "max_length": val_max_target_length,
          "num_beams":4
        }
        if epoch>=0:
            to_dump = []

            for step, batch in enumerate(eval_dataloader):
                with torch.no_grad():

                    generated_tokens = accelerator.unwrap_model(model).generate(
                        batch["input_ids"],
                        attention_mask = batch["attention_mask"],
                        **gen_kwargs
                    )

                    generated_tokens = accelerator.pad_across_processes(
                        generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
                    )

                    labels = batch["labels"]
                    generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
                    labels = accelerator.gather(labels).cpu().numpy()

                    # Replace -100 in the labels as we can't decode them.(ignore pad for loss)
                    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

                    if isinstance(generated_tokens, tuple):
                        generated_tokens = generated_tokens[0]

                    decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
                    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

                    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

                    metric.add_batch(predictions=decoded_preds, references=decoded_labels)

                    decoded_source = tokenizer.batch_decode(batch["input_ids"],skip_special_tokens=True)

                    for source,true,pred in zip(decoded_source,decoded_labels,decoded_preds):
                        sample={"source":source,"true":true,"pred":pred}
                        to_dump.append(sample)

            with open(output_dir+"prediction_epoch{}.json".format(epoch),"w") as f:
                json.dump(to_dump, f,indent=4)

            result = metric.compute(use_stemmer=True)
            # Extract a few results from ROUGE
            result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

            result = {k: round(v, 4) for k, v in result.items()}

            if np.mean(list(result.values())) > best_val_score:
                best_val_score = np.mean(list(result.values()))
                best_epoch = epoch

            print(f'At epoch {epoch}, the evaluation result is {result}, the best score is {best_val_score} at epoch {best_epoch}')

        accelerator.wait_for_everyone()
        unwrapped_model = accelerator.unwrap_model(model)
        os.makedirs(output_dir+f'/{epoch}/', exist_ok=True)
        unwrapped_model.save_pretrained(output_dir+f'/{epoch}', save_function=accelerator.save)

In [8]:
# torch.multiprocessing.set_start_method('spawn')

args = (42, 8)
import accelerate
accelerate.notebook_launcher(training_loop, args, num_processes=2)

Launching training on 2 GPUs.
****************************************************************************************************
****************************************************************************************************
****************************************************************************************************
****************************************************************************************************


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


********************************************************************************************************************************************************************************************************



  0%|          | 0/140 [00:00<?, ?it/s]

****************************************************************************************************
****************************************************************************************************


  6%|▌         | 8/140 [00:26<07:10,  3.26s/it]